#### The final version of the Recommender system demonstrates the "improvement" of the system after receiving user feedback. It uses the LSI model from before, but takes the relevance judgments as feedback to imrpove the next iteration. The only drawback in this scenario is that Rocchio feedback only works with queries that have been seen before. While the system improves greatly, it does nothing to improve the results of the LSI model if the query has not been seen before.

In [857]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from sklearn.decomposition import TruncatedSVD
import json

In [817]:
stop_words = set(stopwords.words("english"))

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n\\b") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(description, stop_words, normalization):
    
    if normalization == 'lemmatize':
        # tokenize and lemmatize text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(w) for w in word_tokenize(description)]
        
    elif normalization == 'stem':
        # tokenize and stem text
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(w) for w in word_tokenize(description)]
    
   # remove tokens length of 2 or below and make all lowercase and remove stop words
    tokens = [w.lower() for w in tokens if (w.lower() not in stop_words) and (len(w) > 2) and(w.isalpha())]
    
    return tokens    
    
def process_query(query, normalization):
    stop_words = set(stopwords.words("english"))
    
    return tokenizer(clean_text(query), stop_words, normalization)

In [818]:
# code example taken from https://towardsdatascience.com/build-a-text-recommendation-system-with-python-e8b95d9f251c
def retrieve_top_n(m, max_docs):
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score \
    mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:max_docs]  
    return best_index

In [819]:
class RocchioRecommenderSystem:
    def __init__(self, docs, num_concepts=100, alpha=1.0, beta=0.75, gamma=0.15):
        self.alpha, self.beta, self.gamma = alpha, beta, gamma
        
        # create a doc-term matrix out of our doc collection
        self.vec = TfidfVectorizer()
        doc_term_mat = self.vec.fit_transform([" ".join(docs[doc_id]) for doc_id in docs])
        self.svd = TruncatedSVD(n_components=num_concepts, random_state=42)
        result = self.svd.fit_transform(doc_term_mat)
        
        self.q_vecs = {}
        
        self.doc_vecs = result # document vectors in a matrix
        
    def retrieve_docs(self, query, max_docs=10, normalization='lemmatize'):
        query = ' '.join(process_query(query, normalization))
        
        if query not in self.q_vecs:
            q_vec = self.vec.transform([query])
            lsi_transform = self.svd.transform(q_vec)
            self.q_vecs[query] = lsi_transform
        
        ret_docs = {}
        
        mat = cosine_similarity(self.q_vecs[query], self.doc_vecs)
        best_index = retrieve_top_n(mat, max_docs=max_docs)
        
        return best_index
    
    def gather_feedback(self, query, max_docs=10, feedback=None, normalization='lemmatize'):
        """
        This function models the interactive relevance feedback loop
        """
        query = ' '.join(process_query(query, normalization))
        # Step 2: Retrieve the required number of docs in reponse to the queries
        ret_docs = self.retrieve_docs(query, max_docs=max_docs)
        
        
        # display docs to user
        # receive feedback from user

        # Step 3: Obtain feedback from the user in the form of precisions at each rank
        # user_feedback = feedback
        # map index to user feedback
        idx_dic = {}
        # for i, doc in enumerate(ret_docs):
        #     try:
        #         idx_dic[doc] = user_feedback[i]
        #     except:
        #         idx_dic[doc] = 0
        
        for feed in feedback:
            if feed[0] == 1:
                idx_dic[feed[1]] = 1
            else:
                idx_dic[feed[1]] = 0
                

        self.q_vecs[query] = np.dot(self.alpha, self.q_vecs[query])
        for key, value in idx_dic.items():
            if value == 1:
                self.q_vecs[query] += np.dot(self.beta, self.doc_vecs[key])
            else:
                self.q_vecs[query] -= np.dot(self.gamma, self.doc_vecs[key])
        self.q_vecs[query][self.q_vecs[query] < 0] = 0

In [820]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))
model = RocchioRecommenderSystem(docs, beta=1.2, gamma=.05)
best_index = model.retrieve_docs('Church', max_docs=100)
naics_titles = pd.read_excel('assets/6-digit_2017_Codes.xlsx')
naics_titles['naics'] = naics_titles['naics'].astype(str)
df = df.merge(naics_titles, on='naics', how='outer')
df.iloc[best_index][['naics', 'title']]

,naics,title
474,337127,Institutional Furniture Manufacturing
470,337121,Upholstered Household Furniture Manufacturing
472,337124,Metal Household Furniture Manufacturing
471,337122,Nonupholstered Wood Household Furniture Manufa...
473,337125,Household Furniture (except Wood and Metal) Ma...
...,...,...
135,238910,Site Preparation Contractors
996,811430,Footwear and Leather Goods Repair
529,423910,Sporting and Recreational Goods and Supplies M...
504,423320,"Brick, Stone, and Related Construction Materia..."


In [821]:
relevant_naics = pd.read_pickle('assets/relevant_naics_df.pkl')

In [822]:
relevant_naics

,query,relevant_naics
0,Home improvement store,"[444110, 444120, 444130, 444190, 444210, 44422..."
1,Diesel fuel supplier,"[424710, 424720, 424110, 424120, 424130, 42421..."
2,Church,"[813110, 813211, 813212, 813219, 813311, 81331..."
3,Farm,"[115116, 115111, 115112, 115113, 115114, 11511..."
4,Seed supplier,"[424910, 424920, 424930, 424940, 424950, 42499..."
...,...,...
1149,State department agricultural development,"[926140, 926110, 926120, 926130, 926150, 92111..."
1150,Military base,"[928110, 928120, 921110, 921120, 921130, 92114..."
1151,Embassy,"[928120, 928110, 921110, 921120, 921130, 92114..."
1152,Window supplier,"[321911, 321912, 321918, 321920, 321991, 32199..."


In [823]:
for row in relevant_naics.iterrows():
    best_index = model.retrieve_docs(row[1]['query'], normalization='lemmatize')
    feedback = []
    for rel_code in row[1]['relevant_naics']:
        try:
            idx = np.where(df['naics'].values == str(rel_code))[0][0]
        except:
            print(idx, rel_code)
        if rel_code in df.iloc[best_index]['naics'].values:
            feedback.append((1, idx))
        else:
            feedback.append((0, idx))
    model.gather_feedback(row[1]['query'], feedback=feedback, max_docs=max_docs, normalization='lemmatize')

In [824]:
df.iloc[model.retrieve_docs('Home improvement store', normalization='lemmatize')][['naics', 'title']]

,naics,title
576,442210,Floor Covering Stores
578,442299,All Other Home Furnishings Stores
577,442291,Window Treatment Stores
575,442110,Furniture Stores
627,453930,Manufactured (Mobile) Home Dealers
629,453998,All Other Miscellaneous Store Retailers (excep...
628,453991,Tobacco Stores
624,453310,Used Merchandise Stores
618,452210,Department Stores
626,453920,Art Dealers


In [825]:
def calc_pre_rec_at_n(ret_docs, reljudges, n=-1):
    """
    Calculate precision and recall at n for each query in ret_docs
    """
    
    pre_at_n, rec_at_n = {}, {}
    
    for k, v in ret_docs.items():
        if n > -1 and n <= len(ret_docs):
            s1 = set(v[:n])
        else:
            s1 = set(v)
        s2 = reljudges[k]
        precision = len(s1.intersection(s2)) / len(s1)
        recall = len(s1.intersection(s2)) / len(reljudges[k])
        pre_at_n[k] = round(precision, 3)
        rec_at_n[k] = round(recall, 3)
    return pre_at_n, rec_at_n

In [826]:
def calc_avg_pre(ret_docs, reljudges, cutoff=-1):
    """
    Calculate (mean) average precision for each query in ret_docs
    """
    
    avg_pre, mean_avg_pre = {}, None
    for k, v in ret_docs.items():
        total_rel = 0
        total = 0
        avg_prec = 0
        for i, doc in enumerate(v):
            if doc in reljudges[k] and cutoff == -1:
                total_rel += 1
                total += 1
                precision = total_rel/total
            elif doc in reljudges[k] and i+1 <= cutoff:
                total_rel += 1
                total += 1
                precision = total_rel/total
            else:
                total += 1
                precision = 0
            avg_prec += precision

        avg_pre[k] = round(avg_prec/len(reljudges[k]), 3,)
    
    mean_avg_pre = round(sum(avg_pre.values()) / len(avg_pre), 3)
        
    return avg_pre, mean_avg_pre

In [827]:
import math

def calc_NDCG_at_n(ret_docs, reljudges, n=-1, base=2):
    """
    Calculate NDCG at n for each query in ret_docs
    """
    
    ndcg = {}
    
    for k, v in ret_docs.items():
        
        counts = list(reversed([x for x in range(2,len(reljudges[k])+2)]))
        ideals = {reljudges[k][i]: counts[i] for i in range(len(reljudges[k]))}
        
        add_ons = {}
        if len(v) > len(reljudges[k]):
            for i in range(len(v)-len(reljudges[k])):
                add_ons[i] = 1
        ideals.update(add_ons)
        nums = list(map(ideals.get, v))
        
        systems = {}
        for i, doc in enumerate(v):
            if nums[i] == None:
                systems[doc] = 1
            else:
                systems[doc] = nums[i]
                
        ideal_order = {}
        if n != -1:
            for i, (key, value) in enumerate(ideals.items()):
                if i < n:
                    ideal_order[key] = value
        else:
            ideal_order = ideals
        
        add_ons = {}
        
        
        log = 0
        for i, (doc, rank) in enumerate(ideal_order.items()):
            if i >= len(v):
                break
            elif i < base:
                log += rank
            else:
                log += rank/math.log(i+1, base)

                
        system_order = {}
        if n != -1:
            for i, (key, value) in enumerate(systems.items()):
                if i < n:
                    system_order[key] = value
        else:
            system_order = systems
        
        
        
        system_log = 0
        for i, (doc, rank) in enumerate(system_order.items()):
            if i >= len(v):
                break
            elif i < base:
                system_log += rank
            else:
                system_log += rank/math.log(i+1, base)
        system_log / log
        ndcg[k] = round(system_log / log, 3)
        
    
    return ndcg

In [828]:
max_docs = 10

In [829]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
    ret_docs_index = model.retrieve_docs(query_name, max_docs=max_docs, normalization='lemmatize')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [830]:
ret_docs_dic

{'home improvement store': ['442210',
  '442299',
  '442291',
  '442110',
  '453930',
  '453998',
  '453991',
  '453310',
  '452210',
  '453920'],
 'diesel fuel supplier': ['332216',
  '332993',
  '332410',
  '332991',
  '332992',
  '332994',
  '332999',
  '332613',
  '332111',
  '332117'],
 'church': ['423310',
  '423520',
  '423210',
  '423390',
  '423510',
  '423940',
  '423320',
  '423990',
  '423330',
  '423910'],
 'farm': ['115210',
  '115115',
  '115113',
  '115114',
  '115116',
  '115112',
  '115111',
  '115310',
  '111310',
  '111320'],
 'seed supplier': ['332216',
  '332410',
  '332999',
  '332215',
  '332993',
  '332991',
  '332992',
  '332994',
  '332111',
  '332439'],
 'accountant': ['541191',
  '541199',
  '541110',
  '541120',
  '541211',
  '541420',
  '541350',
  '541219',
  '541410',
  '541213'],
 'trucking company': ['484122',
  '484110',
  '484121',
  '484220',
  '484230',
  '484210',
  '492210',
  '493110',
  '336214',
  '336212'],
 'exporter': ['522220',
  '522120'

In [831]:
queries_dic

{'home improvement store': ['444110',
  '444120',
  '444130',
  '444190',
  '444210',
  '444220',
  '441110',
  '441120',
  '441210',
  '441222',
  '441228',
  '441310',
  '441320',
  '442110',
  '442210',
  '442291',
  '442299',
  '443141',
  '443142',
  '445110',
  '445120',
  '445210',
  '445220',
  '445230',
  '445291',
  '445292',
  '445299',
  '445310',
  '446110',
  '446120',
  '446130',
  '446191',
  '446199',
  '447110',
  '447190',
  '448110',
  '448120',
  '448130',
  '448140',
  '448150',
  '448190',
  '448210',
  '448310',
  '448320'],
 'diesel fuel supplier': ['424710',
  '424720',
  '424110',
  '424120',
  '424130',
  '424210',
  '424310',
  '424320',
  '424330',
  '424340',
  '424410',
  '424420',
  '424430',
  '424440',
  '424450',
  '424460',
  '424470',
  '424480',
  '424490',
  '424510',
  '424520',
  '424590',
  '424610',
  '424690',
  '424810',
  '424820',
  '424910',
  '424920',
  '424930',
  '424940',
  '424950',
  '424990',
  '423110',
  '423120',
  '423130',
 

In [ ]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improvement store': 0.4,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 1.0,
 'seed supplier': 0.0,
 'accountant': 1.0,
 'trucking company': 0.6,
 'exporter': 1.0,
 'grain elevator': 0.0,
 'popcorn store': 0.9,
 'agricultural service': 0.0,
 'warehouse': 0.4,
 'agricultural production': 1.0,
 'ranch': 1.0,
 'holding company': 0.3,
 'farm equipment supplier': 0.0,
 'store': 1.0,
 'grocery store': 0.9,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'accounting firm': 1.0,
 'produce market': 0.3,
 'pet supply store': 1.0,
 'wholesaler': 1.0,
 'produce wholesaler': 1.0,
 'distribution service': 0.4,
 'crop grower': 1.0,
 'addiction treatment center': 1.0,
 'natural good store': 0.0,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transportation service': 1.0,
 'investment company': 1.0,
 'fruit wholesaler': 1.0,
 'real estate agency': 1.0,
 'event venue': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesaler importer': 1.0,
 'winery': 0.7,
 'aerospace company': 

In [ ]:
# IMPORT INVERTED_INDEXES
with open(r'assets/rocchio-pre.json', 'w') as f:
    json.dump(calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0], f)
    
# IMPORT INVERTED_INDEXES
with open(r'assets/rocchio-re.json', 'w') as f:
    json.dump(calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1], f)

In [ ]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improvement store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.204,
 'trucking company': 0.12,
 'exporter': 0.244,
 'grain elevator': 0.0,
 'popcorn store': 0.205,
 'agricultural service': 0.0,
 'warehouse': 0.571,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.455,
 'grocery store': 0.205,
 'rice mill': 0.137,
 'food product supplier': 0.0,
 'accounting firm': 0.204,
 'produce market': 0.068,
 'pet supply store': 0.455,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.082,
 'crop grower': 0.156,
 'addiction treatment center': 0.256,
 'natural good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.244,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesaler imp

In [ ]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improvement store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.204,
 'trucking company': 0.12,
 'exporter': 0.244,
 'grain elevator': 0.0,
 'popcorn store': 0.205,
 'agricultural service': 0.0,
 'warehouse': 0.571,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.455,
 'grocery store': 0.205,
 'rice mill': 0.137,
 'food product supplier': 0.0,
 'accounting firm': 0.204,
 'produce market': 0.043,
 'pet supply store': 0.455,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.058,
 'crop grower': 0.156,
 'addiction treatment center': 0.256,
 'natural good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.244,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesaler imp

In [ ]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.151

In [ ]:
rocchio_maps = []
for i in range(1, 11):
    rocchio_maps.append(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=i)[1])

In [ ]:
rocchio_maps

[0.017, 0.034, 0.051, 0.066, 0.081, 0.096, 0.11, 0.124, 0.138, 0.151]

In [ ]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improvement store': 0.442,
 'diesel fuel supplier': 0.015,
 'church': 0.021,
 'farm': 0.961,
 'seed supplier': 0.015,
 'accountant': 0.927,
 'trucking company': 0.776,
 'exporter': 0.913,
 'grain elevator': 0.203,
 'popcorn store': 0.949,
 'agricultural service': 0.016,
 'warehouse': 0.839,
 'agricultural production': 0.804,
 'ranch': 0.916,
 'holding company': 0.966,
 'farm equipment supplier': 0.015,
 'store': 0.497,
 'grocery store': 0.916,
 'rice mill': 0.937,
 'food product supplier': 0.015,
 'accounting firm': 0.926,
 'produce market': 0.348,
 'pet supply store': 0.945,
 'wholesaler': 0.659,
 'produce wholesaler': 0.225,
 'distribution service': 0.143,
 'crop grower': 0.86,
 'addiction treatment center': 0.67,
 'natural good store': 0.024,
 'orchard': 0.865,
 'lumber store': 0.015,
 'mine': 0.612,
 'transportation service': 0.129,
 'investment company': 0.997,
 'fruit wholesaler': 0.233,
 'real estate agency': 0.944,
 'event venue': 0.024,
 'frozen dessert supplier': 0.91,

In [843]:
test = model.retrieve_docs('agricultural production', normalization='stem')
df.iloc[test][['naics','title']]

,naics,title
306,327999,All Other Miscellaneous Nonmetallic Mineral Pr...
302,327910,Abrasive Product Manufacturing
303,327991,Cut Stone and Stone Product Manufacturing
305,327993,Mineral Wool Manufacturing
301,327420,Gypsum Product Manufacturing
165,311710,Seafood Product Preparation and Packaging
304,327992,Ground or Treated Mineral and Earth Manufactur...
294,327215,Glass Product Manufacturing Made of Purchased ...
157,311512,Creamery Butter Manufacturing
159,311514,"Dry, Condensed, and Evaporated Dairy Product M..."


In [849]:
relevant_naics[relevant_naics['query'] =='Agricultural production']

,query,relevant_naics
12,Agricultural production,"[111998, 111991, 111992, 111910, 111920, 11193..."


# Stemming

In [807]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['stemmed']))

In [808]:
model = RocchioRecommenderSystem(docs, beta=1.2, gamma=.05)

In [809]:
for row in relevant_naics.iterrows():
    best_index = model.retrieve_docs(row[1]['query'], normalization='stem')
    feedback = []
    for rel_code in row[1]['relevant_naics']:
        idx = np.where(df['naics'].values == str(rel_code))[0][0]
        if rel_code in df.iloc[best_index]['naics'].values:
            feedback.append((1, idx))
        else:
            feedback.append((0, idx))
    model.gather_feedback(row[1]['query'], feedback=feedback, max_docs=max_docs, normalization='stem')

In [810]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'stem'))
    ret_docs_index = model.retrieve_docs(query_name, max_docs=max_docs, normalization='stem')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [811]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improv store': 0.4,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 1.0,
 'seed supplier': 0.0,
 'account': 1.0,
 'truck compani': 0.6,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.9,
 'agricultur servic': 0.0,
 'warehous': 0.4,
 'agricultur product': 1.0,
 'ranch': 1.0,
 'hold compani': 0.3,
 'farm equip supplier': 0.0,
 'store': 0.9,
 'groceri store': 0.9,
 'rice mill': 1.0,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.9,
 'wholesal': 1.0,
 'produc wholesal': 1.0,
 'distribut servic': 0.0,
 'crop grower': 1.0,
 'addict treatment center': 1.0,
 'natur good store': 0.0,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transport servic': 1.0,
 'invest compani': 1.0,
 'fruit wholesal': 0.1,
 'real estat agenc': 1.0,
 'event venu': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesal import': 0.3,
 'wineri': 0.7,
 'aerospac compani': 1.0,
 'cold storag facil': 0.4,
 'employ agenc': 0.0,
 'plant nurseri':

In [812]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improv store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.204,
 'truck compani': 0.12,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.205,
 'agricultur servic': 0.0,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.409,
 'groceri store': 0.205,
 'rice mill': 0.137,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.409,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.0,
 'crop grower': 0.156,
 'addict treatment center': 0.256,
 'natur good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.244,
 'fruit wholesal': 0.014,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesal import': 0.042,
 'wineri': 0.096,
 'aerospac compani': 0.053,
 'cold storag fac

In [813]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improv store': 0.091,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.204,
 'truck compani': 0.12,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.205,
 'agricultur servic': 0.0,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.409,
 'groceri store': 0.205,
 'rice mill': 0.137,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.409,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.0,
 'crop grower': 0.156,
 'addict treatment center': 0.256,
 'natur good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.244,
 'fruit wholesal': 0.002,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesal import': 0.024,
 'wineri': 0.096,
 'aerospac compani': 0.053,
 'cold storag fac

In [814]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.143

In [815]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improv store': 0.442,
 'diesel fuel supplier': 0.015,
 'church': 0.021,
 'farm': 0.842,
 'seed supplier': 0.015,
 'account': 0.957,
 'truck compani': 0.775,
 'export': 0.026,
 'grain elev': 0.203,
 'popcorn store': 0.949,
 'agricultur servic': 0.016,
 'warehous': 0.834,
 'agricultur product': 0.188,
 'ranch': 0.913,
 'hold compani': 0.966,
 'farm equip supplier': 0.015,
 'store': 0.469,
 'groceri store': 0.914,
 'rice mill': 0.978,
 'food product supplier': 0.015,
 'account firm': 0.021,
 'produc market': 0.024,
 'pet suppli store': 0.93,
 'wholesal': 0.315,
 'produc wholesal': 0.744,
 'distribut servic': 0.021,
 'crop grower': 0.864,
 'addict treatment center': 0.604,
 'natur good store': 0.024,
 'orchard': 0.855,
 'lumber store': 0.015,
 'mine': 0.546,
 'transport servic': 0.122,
 'invest compani': 0.972,
 'fruit wholesal': 0.078,
 'real estat agenc': 0.942,
 'event venu': 0.024,
 'frozen dessert supplier': 0.85,
 'wine wholesal import': 0.33,
 'wineri': 0.83,
 'aerospac compa